In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Gen AI Evaluation Service           

In [1]:
# @title Install Vertex AI Python SDK
%pip install --user --quiet google-cloud-aiplatform[evaluation]

Note: you may need to restart the kernel to use updated packages.


In [2]:
# @title Set GCP information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
EXPERIMENT_NAME = "my-eval-task-experiment"  # @param {type:"string"}

In [3]:
# @title Authentication to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [4]:
# @title Initialize Vertex AI SDK
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [17]:
# @title Import libraries
import pandas as pd
pd.set_option('max_colwidth', 1000)


# Main
from vertexai.evaluation import EvalTask, PointwiseMetric, PointwiseMetricPromptTemplate

In [8]:
# @title Evaluation prompt

# Template reference : https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#overview

metric_prompt_template = """

  # Instruction
  You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
  We will provide you with the user input and an AI-generated responses.
  You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
  You will assign the response a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations for your rating, and only choose ratings from the Rating Rubric.

  # Evaluation
  ## Metric Definition
  You will be assessing fluency, which measures language mastery of the model's response based on the user prompt.

  ## Criteria
  Fluency: The text is free of grammatical errors, employs varied sentence structures, and maintains a consistent tone and style, resulting in a smooth and natural flow that is easy to understand.

  ## Rating Rubric
  5: (Completely fluent). The response is free of grammatical errors, demonstrates nuanced word choice, and has a natural, seamless flow.
  4: (Mostly fluent). The response has very few, if any, minor grammatical errors. Word choice is clear, and sentences generally flow well.
  3: (Somewhat fluent). The response has grammatical errors present, which may cause some difficulty for the reader. Word choice is mostly appropriate, but some awkward phrasing or word repetition may exist.
  2: (Somewhat inarticulate). The response has frequent grammatical errors make the writing difficult to understand. Sentence structure is often awkward, and there's little sense of flow.
  1: (Inarticulate). The response is riddled with grammatical issues, rendering it incomprehensible in parts. Word choices may be very limited or inaccurate.

  ## Evaluation Steps
  STEP 1: Assess grammar correctness: Identify any specific errors in the response's sentence structure, verb usage, subject-verb agreement, punctuation, and capitalization.
  STEP 2: Assess word choice and flow: Examine the response's sentence structure and how the writing moves from one idea to the next. Are words accurate and well-suited to the context?
  STEP 3: Assess overall cohesion: Does the entire response read logically and smoothly, with appropriate transitions?

  # User Inputs and AI-generated Response
  ## User Inputs
  ### Prompt
  {prompt}

  ## AI-generated Response
  {response}

"""


In [9]:
text_quality = PointwiseMetric(
    metric="text_quality",
    metric_prompt_template=metric_prompt_template,
)

In [18]:
# @title Prepare evaluation dataset
data = [
    # An example of good text_quality
    ["Life is a rollercoaster, full of ups and downs, but it's the thrill that keeps us coming back for more!", " "],
    # An example of medium text_quality
    ["The weather is nice today, not too hot, not too cold.", " "],
    # An example of poor text_quality
    ["The weather is, you know, whatever.", " "],
]

eval_dataset = pd.DataFrame(data, columns=['prompt', 'response'])
eval_dataset

,prompt,response
0,"Life is a rollercoaster, full of ups and downs, but it's the thrill that keeps us coming back for more!",
1,"The weather is nice today, not too hot, not too cold.",
2,"The weather is, you know, whatever.",


In [11]:
# @title Run evaluation
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[text_quality],
    experiment=EXPERIMENT_NAME
)

eval_result = eval_task.evaluate()

Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0 to Experiment: my-eval-task-experiment


Computing metrics with a total of 3 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 3/3 [00:05<00:00,  1.73s/it]

All 3 metric requests are successfully computed.
Evaluation Took:5.204361282027094 seconds


In [19]:
# @title Results
eval_result.metrics_table

,prompt,response,text_quality/explanation,text_quality/score
0,"Life is a rollercoaster, full of ups and downs, but it's the thrill that keeps us coming back for more!",,"STEP 1: Assess grammar correctness: The response is an empty string and therefore does not contain any grammatical errors.\nSTEP 2: Assess word choice and flow: There are no words or sentences, so no assessment can be made regarding word choice and flow.\nSTEP 3: Assess overall cohesion: There is no content to assess for cohesion.\n\nOverall, due to the empty nature of the response, I cannot assign a rating based on the provided rubric. If I had to make a choice, it would be 1, but that implies grammatical errors, which is untrue here. If the system allows for such a deviation, it would be 1.",1.0
1,"The weather is nice today, not too hot, not too cold.",,"STEP 1: Assess grammar correctness: The response is empty, so no grammatical errors are present.\nSTEP 2: Assess word choice and flow: Since the response is empty, there are no words to assess. There is no flow to analyze.\nSTEP 3: Assess overall cohesion: The emptiness of the response renders it impossible to determine whether the generated response is cohesive. Since it does not provide anything, it cannot be evaluated on its fluency. Although no grammatical errors are present, I scored the response with a 1 due to a lack of any relevant content.\n",1.0
2,"The weather is, you know, whatever.",,"STEP 1: Assess grammar correctness: The response is a sentence fragment and is not grammatically correct.\nSTEP 2: Assess word choice and flow: The word choice is simplistic and vague, offering no substantive information, and there is no discernable flow as it is just a short phrase.\nSTEP 3: Assess overall cohesion: The response lacks any cohesion, failing to convey a clear or complete thought. \nTherefore, the response's fluency earns a rating of 1 because it is difficult to understand due to grammatical errors and poor sentence structure. It uses simplistic language and is not a complete sentence.",1.0


In [13]:
# @title Clean up
from google.cloud import aiplatform

aiplatform.ExperimentRun(
    run_name=eval_result.metadata["experiment_run"],
    experiment=eval_result.metadata["experiment"],
).delete()

Experiment run 0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0 skipped backing tensorboard run deletion.
To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"my-eval-task-experiment-0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()
Deleting Context : projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0
Context deleted. . Resource name: projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0
Deleting Context resource: projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-0a3797a3-9e0f-490f-ae16-9e1ab6bcfdb0
Delete Context backing LRO: projec